In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
Root = "/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio"
os.chdir(Root)

In [2]:
ls

drive/  sample_data/


In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [31]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [32]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [33]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/archive (7)/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [34]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.2)

In [35]:
x_train

array([[-6.05290344e+02,  8.02614441e+01,  7.76620054e+00, ...,
         3.21403386e-05,  1.97774807e-05,  1.07643182e-05],
       [-5.43856812e+02,  7.62956619e+01, -7.34181023e+00, ...,
         2.04878888e-05,  1.28838756e-05,  1.45694712e-05],
       [-4.15215027e+02,  2.43522663e+01, -8.86893082e+00, ...,
         6.97721774e-03,  5.49639948e-03,  2.36707414e-03],
       ...,
       [-7.95604980e+02,  3.55429993e+01,  1.65204525e+00, ...,
         6.26254950e-06,  4.75230536e-06,  2.51864572e-06],
       [-5.11510010e+02,  4.20126495e+01,  3.37252593e+00, ...,
         6.06912305e-04,  5.72417746e-04,  4.06772306e-04],
       [-6.62019653e+02,  4.55445366e+01, -3.82717562e+00, ...,
         1.04338724e-04,  4.46671038e-05,  2.33882092e-05]])

In [36]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(614, 154)


In [38]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [110]:
model = MLPClassifier(alpha=0.001, batch_size=128, epsilon=1e-08, hidden_layer_sizes=(300, ), learning_rate='adaptive', max_iter=1000)


In [111]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.001, batch_size=128, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=1000)

In [112]:
#Predict for the test set
y_pred=model.predict(x_test)

In [113]:
y_pred

array(['happy', 'calm', 'disgust', 'calm', 'calm', 'happy', 'happy',
       'calm', 'fearful', 'disgust', 'happy', 'calm', 'calm', 'calm',
       'calm', 'fearful', 'fearful', 'fearful', 'calm', 'happy',
       'disgust', 'calm', 'disgust', 'happy', 'fearful', 'calm', 'calm',
       'calm', 'disgust', 'disgust', 'calm', 'happy', 'fearful', 'happy',
       'fearful', 'fearful', 'happy', 'happy', 'calm', 'happy', 'happy',
       'calm', 'disgust', 'happy', 'calm', 'happy', 'fearful', 'fearful',
       'happy', 'fearful', 'disgust', 'happy', 'calm', 'calm', 'calm',
       'disgust', 'calm', 'fearful', 'happy', 'disgust', 'disgust',
       'calm', 'fearful', 'calm', 'calm', 'disgust', 'fearful', 'calm',
       'happy', 'disgust', 'happy', 'calm', 'fearful', 'disgust',
       'fearful', 'happy', 'disgust', 'disgust', 'calm', 'happy', 'happy',
       'calm', 'disgust', 'happy', 'disgust', 'disgust', 'disgust',
       'fearful', 'happy', 'happy', 'calm', 'calm', 'fearful', 'fearful',
       '

In [114]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 75.97%


In [115]:
from sklearn.metrics import accuracy_score, f1_score

In [116]:
f1_score(y_test, y_pred,average=None)

array([0.87179487, 0.72289157, 0.78947368, 0.64788732])

In [117]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,happy
1,disgust,calm
2,fearful,disgust
3,calm,calm
4,calm,calm
5,happy,happy
6,fearful,happy
7,calm,calm
8,fearful,fearful
9,disgust,disgust


In [118]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [123]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/TESS Toronto emotional speech set data/YAF_sad/YAF_bar_sad.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [120]:
feature

array([[-4.04107117e+02,  8.71637955e+01,  1.62905655e+01,
         2.25893784e+01,  1.92002869e+00,  2.09869099e+00,
         2.07418442e+00, -1.49927692e+01,  2.36412311e+00,
        -2.74010420e+00, -1.35020494e+01,  5.26167870e+00,
        -1.12806253e+01,  2.76523995e+00,  7.05204129e-01,
        -8.84847355e+00, -1.19785674e-01, -9.35018182e-01,
        -4.64407444e+00, -7.03576899e+00, -5.94885492e+00,
        -5.45878267e+00, -9.02057362e+00, -1.93593752e+00,
        -2.92647171e+00, -1.53615281e-01,  5.13150883e+00,
         3.71534514e+00, -2.29936630e-01,  1.29668677e+00,
        -1.00574934e+00,  2.38466144e+00,  9.34167099e+00,
         8.61570930e+00,  6.21770287e+00,  3.21086764e+00,
        -2.73324668e-01,  3.78035426e+00,  4.51885414e+00,
         8.60358810e+00,  5.33995688e-01,  4.96827036e-01,
         3.85391533e-01,  3.69037777e-01,  3.96507233e-01,
         4.64605629e-01,  4.44120765e-01,  4.21688676e-01,
         5.17207682e-01,  4.90575939e-01,  3.97035867e-0